# 問題: 飛行機の遅延を予測する

このノートブックの各種目標は以下のとおりです。
- ダウンロードした ZIP ファイルを処理してデータセットを作成する
- 探索的データ分析 (EDA) を行う
- ベースラインモデルを確立する
- シンプルなモデルからアンサンブルモデルに移行する
- ハイパーパラメータを最適化する
- 特徴量の重要度を確認する

## ビジネスシナリオの概要
あなたは勤務先の会社で、旅行予約ウェブサイトの運営を担当しています。このウェブサイトでは、フライトが遅延した場合のカスタマーエクスペリエンスの向上に取り組んでいます。自社では、米国の国内線で発着便数の非常に多い空港を発着するフライトを予約している顧客向けに、天候によりフライトが遅延するかどうかを知らせる機能を作成したいと考えています。

今回のタスクは、天候によりフライトが遅延するかどうかを機械学習で特定し、この問題の一端を解決することです。データセットは、大手航空会社が運航する国内線の定刻パフォーマンスに関するものを使用します。このデータを使用して機械学習モデルをトレーニングし、発着便数の非常に多い空港でフライトが遅延するかどうかを予測します。

## このデータセットについて
データセットには、予定離着陸時刻と実際の離着陸時刻が含まれます。これは、米国内の予定旅客輸送収益の 1% 以上を占める、米国の認可航空会社から報告されたものです。データは、米国運輸統計局 (BTS) の航空情報庁が収集しました。データセットは、2013 年から 2018 年のフライトの日付、飛行時間、出発地、目的地、航空会社、飛行距離、遅延ステータスで構成されます。

### 特徴量
データセットの特徴量の詳細については、[On-time delay dataset features](https://www.transtats.bts.gov/Fields.asp) を参照してください。

### データセットの属性  
ウェブサイト: https://www.transtats.bts.gov/

このラボで使用するデータセットは、米国運輸統計局 (BTS) の航空情報庁でまとめられたものであり、航空機の定刻パフォーマンスデータは https://www.transtats.bts.gov/DatabaseInfo.asp?DB_ID=120&amp;DB_URL=Mode_ID=1&amp;Mode_Desc=Aviation&amp;Subject_ID2=0 で確認できます。

# ステップ 1: 問題の定式化とデータ収集

このプロジェクトを始めるにあたり、このシナリオにおけるビジネス上の問題と達成しようとしているビジネス目標を 2、3 文にまとめて、以下に入力します。これには、チームが目指す必要のあるビジネスメトリクスを含めます。その情報を定義したら、機械学習の問題文を明確に書き出します。最後に、これが表す機械学習のタイプに関するコメントを 1、2 文追加します。

#### <span style="color: blue;">プロジェクトプレゼンテーション: これらの詳細の要約をプロジェクトプレゼンテーションに含めます。</span>

### 1.機械学習がデプロイすべき適切なソリューションかどうか、また適切なソリューションである場合はその理由を判断します。

In [ ]:
# Write your answer here

### 2.ビジネス上の問題、成功のメトリクス、求める機械学習の出力を定式化します。

In [ ]:
# Write your answer here

### 3.取り組んでいる機械学習の問題のタイプを特定します。

In [ ]:
# Write your answer here

### 4.使用しているデータの適切性を分析します。

In [ ]:
# Write your answer here

### 設定

注力する領域を特定したところで、問題を解決するための作業を開始できるようにセットアップしましょう。

**注意:** このノートブックは `ml.m4.xlarge` ノートブックインスタンスで作成およびテストされました。

In [ ]:
import os
from pathlib2 import Path
from zipfile import ZipFile
import time

import pandas as pd
import numpy as np
import subprocess

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# ステップ 2: データの前処理と可視化  
このデータの前処理フェーズでは、データを探索して可視化し、データに対する理解を深める必要があります。まず、必要なライブラリをインポートし、データを pandas の DataFrame に読み込みます。その後、データを探索します。データセットのシェイプを確認し、作業している列と列のタイプ (数値、カテゴリ) を調べます。特徴量に対して基本的な統計を実行して、特徴量の平均と範囲を理解することを検討します。ターゲット列を詳細に調べて、その分布を判断します。

### 考慮すべき具体的な質問
1.特徴量に対して実行した基本的な統計からどのようなことを推測できますか? 

2.ターゲットクラスの分布から、どのようなことを推測できますか?

3.データを探索することで推測できたことは他にありますか?

#### <span style="color: blue;">プロジェクトプレゼンテーション: これらの質問や他の同様の質問に対する自分の回答の要約をプロジェクトプレゼンテーションに含めます。</span>

まず、Amazon S3 パブリックバケットからこのノートブック環境にデータセットを取り込みます。

In [ ]:
# Check whether the file is already in the desired path or if it needs to be downloaded

base_path = '/home/ec2-user/SageMaker/project/data/FlightDelays/'
csv_base_path = '/home/ec2-user/SageMaker/project/data/csvFlightDelays/'
file_path = 'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2014_1.zip'

if not os.path.isfile(base_path + file_path):
    subprocess.run(['mkdir', '-p', base_path])
    subprocess.run(['mkdir', '-p', csv_base_path])
    subprocess.run(['aws', 's3', 'cp',
                    's3://aws-tc-largeobjects/ILT-TF-200-MLDWTS/flight_delay_project/csvFlightData-5/',
                    base_path,'--recursive'])
else:
    print('File already downloaded!')

In [ ]:
zip_files = [str(file) for file in list(Path(base_path).iterdir()) if '.zip' in str(file)]
len(zip_files)

#### ZIP ファイルから CSV ファイルを抽出する

In [ ]:
def zip2csv(zipFile_name , file_path = '/home/ec2-user/SageMaker/project/data/csvFlightDelays'):
    """
    Extract csv from zip files
    zipFile_name: name of the zip file
    file_path : name of the folder to store csv
    """
    try:
        with ZipFile(zipFile_name, 'r') as z: 
            print(f'Extracting {zipFile_name} ') 
            z.extractall(path=file_path) 
    except:
        print(f'zip2csv failed for {zipFile_name}')

for file in zip_files:
    zip2csv(file)

print("Files Extracted")

In [ ]:
csv_files = [str(file) for file in list(Path(csv_base_path).iterdir()) if '.csv' in str(file)]
len(csv_files)

CSV ファイルをロードする前に、展開したフォルダから HTML ファイルを読み取ります。この HTML ファイルに、データセットに含まれる特徴量の背景情報と詳細情報が含まれています。

In [ ]:
from IPython.display import IFrame

IFrame(src="./data/csvFlightDelays/readme.html", width=1000, height=600)

#### サンプル CSV をロードする

すべての CSV ファイルを結合する前に、CSV ファイルごとにデータを読み取ります。pandas を使って、まず `On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2018_9.csv` を読み取ります。Python の組み込みの `read_csv` 関数 ([ドキュメント](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)) を使用できます。

In [ ]:
df_temp = pd.read_csv(<CODE> # **ENTER YOUR CODE HERE**

**問題**: データセット内の行と列の長さ、および列名を出力してください。

**ヒント**: DataFrame の行と列の出力には `<dataframe>.shape` 関数を使用し、列名の出力には `<dataframe>.columns` 関数を使用します。

In [ ]:
df_shape = # **ENTER YOUR CODE HERE**
print(f'Rows and columns in one csv file is {df_shape}')

**問題**: データセットの最初の 10 行を出力してください。 

**ヒント**: `x` 個の行の出力には pandas の組み込み関数 `head(x)` を使用します。

In [ ]:
# Enter your code here

**問題**: データセット内の列をすべて出力してください。`<dataframe>.columns` を使用して列名を出力します。

In [ ]:
print(f'The column names are :')
print('#########')
for col in <CODE>:# **ENTER YOUR CODE HERE**
    print(col)

**問題**: データセットから、'Del' という単語を含む列をすべて出力してください。この操作によって、遅延データがある列の数を確認できます。

**ヒント**: Python のリストに関する理解に基づいて、`if` ステートメントに特定の条件を指定する値を含めることができます。

例: `[x for x in [1,2,3,4,5] if x > 2]`  

**ヒント**: `in` キーワード ([ドキュメント](https://www.w3schools.com/python/ref_keyword_in.asp)) を使用して、値がリストにあるかどうかを確認できます。

例: `5 in [1,2,3,4,5]`

In [1]:
# Enter your code here

データセットの分析に役立つ質問を以下にいくつか紹介します。

**問題**   
1.データセットに行と列はそれぞれいくつありますか?   
2.データセットに何年分のデータが含まれていますか?   
3.データセットが対象とする期間はどのくらいですか?   
4.データセットにどの航空会社が含まれていますか?   
5.どの発着空港が対象となっていますか?

In [ ]:
print("The #rows and #columns are ", <CODE> , " and ", <CODE>)
print("The years in this dataset are: ", <CODE>)
print("The months covered in this dataset are: ", <CODE>)
print("The date range for data is :" , min(<CODE>), " to ", max(<CODE>))
print("The airlines covered in this dataset are: ", list(<CODE>))
print("The Origin airports covered are: ", list(<CODE>))
print("The Destination airports covered are: ", list(<CODE>))

**問題**: 発着空港の総数はいくつですか?

**ヒント**: pandas の `values_count` 関数 ([ドキュメント](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html)) を使って、列 `Origin` と `Dest` で各空港の値を確認できます。

In [ ]:
counts = pd.DataFrame({'Origin':<CODE>, 'Destination':<CODE>})
counts

**問題**: データセット内のフライト数に基づいて上位 15 件の発着空港を出力してください。

**ヒント**: pandas の `sort_values` 関数 ([ドキュメント](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html)) を使用できます。

In [ ]:
counts.sort_values(by=<CODE>,ascending=False).head(15 )# Enter your code here

**問題**: フライトに関するすべての情報に基づいて、そのフライトが遅延するかどうかを予測できますか?

In [ ]:
# Enter your answer here

例えば、出張でサンフランシスコからロサンゼルスへ向かうとします。ロサンゼルスでの予約を効率良く管理できるよう、一連の特徴量に基づき、フライトが遅延するかどうかを知りたいと考えています。このデータセットには、フライトの前に確認できる特徴量がいくつありますか?

`DepDelay`、`ArrDelay`、`CarrierDelay`、`WeatherDelay`、`NASDelay`、`SecurityDelay`、`LateAircraftDelay`、`DivArrDelay` などの列に、遅延に関する情報が含まれています。ただし、この遅延は出発地と目的地のいずれかで発生しました。着陸 10 分前に天候の急変で遅延が発生した場合、このデータはロサンゼルスでの予約を管理するうえでは役立ちません。

そこで、問題文をシンプルにするため、以下の列に基づいて到着時の遅延を予測します。<br>

`Year`、`Quarter`、`Month`、`DayofMonth`、`DayOfWeek`、`FlightDate`、`Reporting_Airline`、`Origin`、`OriginState`、`Dest`、`DestState`、`CRSDepTime`、`DepDelayMinutes`、`DepartureDelayGroups`、`Cancelled`、`Diverted`、`Distance`、`DistanceGroup`、`ArrDelay`、`ArrDelayMinutes`、`ArrDel15`、`AirTime`

また、以下のように発着空港をフィルタリングします。
- 主要空港: ATL、ORD、DFW、DEN、CLT、LAX、IAH、PHX、SFO
- トップ 5 の航空会社: UA、OO、WN、AA、DL

この操作を行うと、結合する CSV ファイル全体のデータのサイズを縮小できます。

#### すべての CSV ファイルを結合する

**ヒント**:  
まず、各ファイルから個々の DataFrame をコピーするために使用する空の DataFrame を作成します。次に、`csv_files` リスト内のファイルごとに以下の操作を行います。

1.DataFrame に CSV ファイルを読み込みます。  
2.`filter_cols` 変数に基づいて列をフィルタリングします。

```
        columns = ['col1', 'col2']
        df_filter = df[columns]
```

3.各 subset_cols に subset_vals のみを残します。pandas の `isin` 関数 ([ドキュメント](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html)) を使用して、`val` が DataFrame の列にあるかどうかを確認し、それを含む行を選択します。

```
        df_eg[df_eg['col1'].isin('5')]
```

4.DataFrame を空の DataFrame と連結します。 

In [ ]:
def combine_csv(csv_files, filter_cols, subset_cols, subset_vals, file_name = 'data/combined_files.csv'):
    """
    Combine csv files into one Data Frame
    csv_files: list of csv file paths
    filter_cols: list of columns to filter
    subset_cols: list of columns to subset rows
    subset_vals: list of list of values to subset rows
    """
    # Create an empty dataframe
    df = # Enter your code here 
    
    for file in csv_files:
        # Read the CSV file into a dataframe
        df_temp = pd.read_csv(<CODE>)# Enter your code here
        
        # Filter the columns based on the filter_cols variable
        # e.g. columns = ['col1', 'col2']
        # df_filter = df[columns]
        df_temp = # Enter your code here
        
        # Keep only the subset_vals in each of the subset_cols
        # HINT: Use the `isin` function to check if the val is in dataframe column
        # and then choose the rows that include it
        # e.g. df[df['col1'].isin('5')]
        for col, val in zip(subset_cols,subset_vals):
            df_temp = # Enter your code here     
        
        # Use Pandas concatenate `pd.concat` to concatenate the main dataframe with the dataframe for each file
        df = pd.concat([df, df_temp], axis=0)
    
        
    df.to_csv(file_name, index=False)
    print(f'Combined csv stored at {file_name}')

In [ ]:
#cols is the list of columns to predict Arrival Delay 
cols = ['Year','Quarter','Month','DayofMonth','DayOfWeek','FlightDate',
        'Reporting_Airline','Origin','OriginState','Dest','DestState',
        'CRSDepTime','Cancelled','Diverted','Distance','DistanceGroup',
        'ArrDelay','ArrDelayMinutes','ArrDel15','AirTime']

subset_cols = ['Origin', 'Dest', 'Reporting_Airline']

# subset_vals is a list collection of the top origin and destination airports and top 5 airlines
subset_vals = [['ATL', 'ORD', 'DFW', 'DEN', 'CLT', 'LAX', 'IAH', 'PHX', 'SFO'],
               ['ATL', 'ORD', 'DFW', 'DEN', 'CLT', 'LAX', 'IAH', 'PHX', 'SFO'],
               ['UA', 'OO', 'WN', 'AA', 'DL']]

上記の関数を使用して各種ファイルをすべて単一のファイルにマージして、簡単に読み取れるようにします。

**注意**: この処理が完了するまで 5～7 分かかります。

In [ ]:
start = time.time()
combine_csv(csv_files, cols, subset_cols, subset_vals)
print(f'csv\'s merged in {round((time.time() - start)/60,2)} minutes')

#### データセットをロードする

結合されたデータセットをロードします。

In [ ]:
data = pd.read_csv(<CODE>)# Enter your code here to read the combined csv file.

最初のレコード 5 つを出力します。

In [ ]:
# Enter your code here 

データセットの分析に役立つ質問を以下にいくつか紹介します。

**問題**   
1.データセットに行と列はそれぞれいくつありますか?   
2.データセットに何年分のデータが含まれていますか?   
3.データセットが対象とする期間はどのくらいですか?   
4.データセットにどの航空会社が含まれていますか?   
5.どの発着空港が対象となっていますか?

In [ ]:
print("The #rows and #columns are ", <CODE> , " and ", <CODE>)
print("The years in this dataset are: ", list(<CODE>))
print("The months covered in this dataset are: ", sorted(list(<CODE>)))
print("The date range for data is :" , min(<CODE>), " to ", max(<CODE>))
print("The airlines covered in this dataset are: ", list(<CODE>))
print("The Origin airports covered are: ", list(<CODE>))
print("The Destination airports covered are: ", list(<CODE>))

では、**target column : is_delay** を定義しましょう (1 - 到着時刻の遅延が 15 分を超える場合、0 - それ以外の場合)。`rename` メソッドを使用して、列名を `ArrDel15` から `is_delay` に変更します。

**ヒント**: pandas の `rename` 関数 ([ドキュメント](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html)) を使用できます。

例:
```
df.rename(columns={'col1':'column1'}, inplace=True)
```

In [ ]:
data.rename(columns=<CODE>, inplace=True) # Enter your code here

列全体で null を検索します。`isnull()` 関数 ([ドキュメント](https://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.isnull.html)) を使用できます。

**ヒント**: `isnull()` では、特定の値が null かどうかが検出され、null の代わりにブール値 (True または False) が返されます。`sum(axis=0)` 関数を使用して列の数を合計します。

In [ ]:
# Enter your code here

1658130 行のうち、1.3% に当たる 22540 行に、到着時の遅延情報と飛行時間が含まれていません。これらの行を削除または補完できます。ドキュメントには、行の欠落については一切記載されていません。

**ヒント**: `~` 演算子を使用して、`isnull()` の出力から null 以外の値を選択します。

例:
```
null_eg = df_eg[~df_eg['column_name'].isnull()]
```

In [ ]:
### Remove null columns
data = # Enter your code here

CRSDepTime から 24 時間形式の時刻を取得します。

In [ ]:
data['DepHourofDay'] = # Enter your code here

## **機械学習の問題文**
- 一連の特徴量に基づいて、フライトの遅延が 15 分を超えるかどうかを予測できますか?
- ターゲット変数に入る値は 0/1 のみのため、分類アルゴリズムを使用できます。

モデリングに進む前に、特徴量の分散、相関などを必ず確認するようお勧めします。
- これで、データの非線形/パターンがすべてわかる
    - 線形モデル: 検出力/指数/相互作用の特徴量を追加する
    - 非線形モデルを試す
- データの不均衡 
    - モデルパフォーマンスに偏りが生じないメトリクスを選択する (精度と AUC の比較)
    - 加重/カスタム損失関数を使用する
- データの欠落
    - 単純な統計に基づいて補完を行う - 平均値、中央値、最頻値 (数値変数)、頻度クラス (カテゴリ変数)
    - クラスターベースの補完 (KNN で列の値を予測)
    - 列を削除する

### データ探索

#### クラスでの遅延ありと遅延なしを比較する

**ヒント**: `groupby` プロット ([ドキュメント](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html)) と `bar` プロット ([ドキュメント](https://matplotlib.org/tutorials/introductory/pyplot.html)) を使用して、クラスの頻度と分散のグラフを作成します。

In [ ]:
(data.groupby(<CODE>).size()/len(data) ).plot(kind='bar')# Enter your code here
plt.ylabel('Frequency')
plt.title('Distribution of classes')
plt.show()

**問題**: 遅延ありと遅延なしの比率に関する棒グラフからどのようなことを推測できますか?

In [ ]:
# Enter your answer here

**問題**: 

- 遅延が最も多いのは何月ですか?
- 遅延が最も多いのは何時ですか?
- 遅延が最も多いのは何曜日ですか?
- 遅延が最も多いのはどの航空会社ですか?
- 遅延が最も多い発着空港はどこですか?
- 飛行距離は遅延の要因ですか?

In [ ]:
viz_columns = ['Month', 'DepHourofDay', 'DayOfWeek', 'Reporting_Airline', 'Origin', 'Dest']
fig, axes = plt.subplots(3, 2, figsize=(20,20), squeeze=False)
# fig.autofmt_xdate(rotation=90)

for idx, column in enumerate(viz_columns):
    ax = axes[idx//2, idx%2]
    temp = data.groupby(column)['is_delay'].value_counts(normalize=True).rename('percentage').\
    mul(100).reset_index().sort_values(column)
    sns.barplot(x=column, y="percentage", hue="is_delay", data=temp, ax=ax)
    plt.ylabel('% delay/no-delay')
    

plt.show()

In [ ]:
sns.lmplot( x="is_delay", y="Distance", data=data, fit_reg=False, hue='is_delay', legend=False)
plt.legend(loc='center')
plt.xlabel('is_delay')
plt.ylabel('Distance')
plt.show()

In [ ]:
# Enter your answers here

### 特徴量

列と、列の具体的なタイプをすべて確認します。

In [ ]:
data.columns

必須の列を以下のようにフィルタリングします。
- 日付を表す列には Year、Quarter、Month、DayofMonth、DayOfWeek があるため、日付は冗長です。
- OriginState と DestState の代わりに Origin コードと Dest コードを使用します。
- フライトが遅延するかどうかを分類するだけであるため、TotalDelayMinutes、DepDelayMinutes、ArrDelayMinutes は必要ありません。

DepHourofDay はターゲットとの定量的関係がないため、カテゴリ変数として扱います。
- ワンホットエンコーディングを行う必要がある場合は、23 列増えます。
- カテゴリ変数を扱うその他の方法として、ハッシュエンコーディング、平均値正規化エンコーディング、値のバケット化などがあります。
- ここでは単にバケットに分割します。

**ヒント**: 列タイプをカテゴリに変更するには、`astype` 関数 ([ドキュメント](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html)) を使用します。

In [ ]:
data_orig = data.copy()
data = data[[ 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Reporting_Airline', 'Origin', 'Dest','Distance','DepHourofDay', 'is_delay']]
categorical_columns = ['Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Reporting_Airline', 'Origin', 'Dest', 'DepHourofDay', 'is_delay']
for c in categorical_columns:
    data[c] = data[c].astype('category')# Enter your code here

ワンホットエンコーディングを使用するには、上記で選択したカテゴリ列に pandas の `get_dummies` 関数を使用します。次に、pandas の `concat` 関数を使用して、生成した特徴量を連結して元のデータセットに戻すことができます。カテゴリ変数のエンコーディングには、キーワード `drop_first=True` を使用して**ダミーエンコーディング**を使うこともできます。ダミーエンコーディングの詳細については、https://en.wikiversity.org/wiki/Dummy_variable_(statistics) を参照してください。

例:
```
pd.get_dummies(df[['column1','columns2']], drop_first=True)
```

In [ ]:
data_dummies = pd.get_dummies(<CODE>, drop_first=True) # Enter your code here
data = pd.concat([<CODE>, <CODE>], axis = 1)
categorical_columns.remove('is_delay')
data.drop(categorical_columns,axis=1, inplace=True)

データセットの長さと新しい列を確認します。

In [ ]:
# Enter your code here

In [ ]:
# Enter your code here

**解答例:** 
```
Index(['Distance', 'is_delay', 'Quarter_2', 'Quarter_3', 'Quarter_4',
       'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6', 'Month_7',
       'Month_8', 'Month_9', 'Month_10', 'Month_11', 'Month_12',
       'DayofMonth_2', 'DayofMonth_3', 'DayofMonth_4', 'DayofMonth_5',
       'DayofMonth_6', 'DayofMonth_7', 'DayofMonth_8', 'DayofMonth_9',
       'DayofMonth_10', 'DayofMonth_11', 'DayofMonth_12', 'DayofMonth_13',
       'DayofMonth_14', 'DayofMonth_15', 'DayofMonth_16', 'DayofMonth_17',
       'DayofMonth_18', 'DayofMonth_19', 'DayofMonth_20', 'DayofMonth_21',
       'DayofMonth_22', 'DayofMonth_23', 'DayofMonth_24', 'DayofMonth_25',
       'DayofMonth_26', 'DayofMonth_27', 'DayofMonth_28', 'DayofMonth_29',
       'DayofMonth_30', 'DayofMonth_31', 'DayOfWeek_2', 'DayOfWeek_3',
       'DayOfWeek_4', 'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_7',
       'Reporting_Airline_DL', 'Reporting_Airline_OO', 'Reporting_Airline_UA',
       'Reporting_Airline_WN', 'Origin_CLT', 'Origin_DEN', 'Origin_DFW',
       'Origin_IAH', 'Origin_LAX', 'Origin_ORD', 'Origin_PHX', 'Origin_SFO',
       'Dest_CLT', 'Dest_DEN', 'Dest_DFW', 'Dest_IAH', 'Dest_LAX', 'Dest_ORD',
       'Dest_PHX', 'Dest_SFO'],
      dtype='object')
```

これで、モデルトレーニングを行う準備ができました。データを分割する前に、列の名前を `is_delay` から `target` に変更します。

**ヒント**: pandas の `rename` 関数 ([ドキュメント](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html)) を使用できます。

In [ ]:
data.rename(columns = {<CODE>:<CODE>}, inplace=True )# Enter your code here

## <span style="color:red">ラボ 2 の終わり</span>

ローカルコンピュータにプロジェクトファイルを保存します。次の一連のステップを実行します。

1.ページ上部にある [**File**] メニューをクリックします。

1.[**Download as**] を選択し、[**Notebook(.ipynb)**] をクリックします。 

これにより、現在のノートブックがコンピュータのデフォルトのダウンロードフォルダにダウンロードされます。

# ステップ 3: モデルのトレーニングと評価

データセットを DataFrame から機械学習アルゴリズムで使用できる形式に変換するときに、実行する必要がある準備手順があります。Amazon SageMaker の場合、以下の手順を実行する必要があります。

1.`sklearn.model_selection.train_test_split` を使用し、データを `train_data`、`validation_data`、`test_data` に分割します。 
2.Amazon SageMaker トレーニングジョブで使用できる適切なファイル形式にデータセットを変換します。これは CSV ファイルまたは record protobuf のいずれかです。詳細については、[トレーニングの共通データ形式](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html) を参照してください。 
3.Amazon S3 バケットにデータをアップロードします。バケットを作成したことがない場合は、[バケットの作成](https://docs.aws.amazon.com/AmazonS3/latest/gsg/CreatingABucket.html) を参照してください。 

以下のセルを使用して、これらの手順を完了します。必要に応じてセルを挿入および削除します。

#### <span style="color: blue;">プロジェクトプレゼンテーション: プロジェクトプレゼンテーションに、このフェーズでの主な決定事項を記録します。</span>

### トレーニングとテストの分割

In [ ]:
from sklearn.model_selection import train_test_split

def create_training_sets(data):
    """
    Convert data frame to train, validation and test
    params:
        data: The dataframe with the dataset to be split
    Returns:
        train_features: Training feature dataset
        test_features: Test feature dataset 
        train_labels: Labels for the training dataset
        test_labels: Labels for the test dataset
        val_features: Validation feature dataset
        val_labels: Labels for the validation dataset
    """
    # Extract the target variable from the dataframe and convert the type to float32
    ys = np.array(<CODE>).astype("float32") # Enter your code here
    
    # Drop all the unwanted columns including the target column
    drop_list = # Enter your code here
    
    # Drop the columns from the drop_list and convert the data into a NumPy array of type float32
    xs = np.array(data.drop(<CODE>, axis=1)).astype("float32")# Enter your code here
    
    np.random.seed(0)

    # Use the sklearn function train_test_split to split the dataset in the ratio train 80% and test 20%
    # Example: train_test_split(x, y, test_size=0.3)
    train_features, test_features, train_labels, test_labels = # Enter your code here
    
    # Use the sklearn function again to split the test dataset into 50% validation and 50% test
    val_features, test_features, val_labels, test_labels = # Enter your code here
    
    
    return train_features, test_features, train_labels, test_labels, val_features, val_labels

In [ ]:
# Use the function to create your datasets
train_features, test_features, train_labels, test_labels, val_features, val_labels = create_training_sets(data)

print(f"Length of train_features is: {<CODE>}")
print(f"Length of train_labels is: {<CODE>}")
print(f"Length of val_features is: {<CODE>}")
print(f"Length of val_labels is: {<CODE>}")
print(f"Length of test_features is: {<CODE>}")
print(f"Length of test_labels is: {<CODE>}")

**解答例**
```
Length of train_features is: (1308472, 71)
Length of train_labels is: (1308472,)
Length of val_features is: (163559, 71)
Length of val_labels is: (163559,)
Length of test_features is: (163559, 71)
Length of test_labels is: (163559,)
```

### ベースライン分類モデル

In [ ]:
import sagemaker
from sagemaker.predictor import CSVSerializer
from sagemaker.amazon.amazon_estimator import RecordSet
import boto3

num_classes = # Enter your code here

# Instantiate the LinearLearner estimator object with 1 ml.m4.xlarge
classifier_estimator = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                               instance_count=<CODE>,
                                               instance_type=<CODE>,
                                               predictor_type=<CODE>,
                                              binary_classifier_model_selection_criteria=<CODE>)

### サンプルコード
```
num_classes = len(pd.unique(train_labels))
classifier_estimator = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                              instance_count=1,
                                              instance_type='ml.m4.xlarge',
                                              predictor_type='binary_classifier',
                                             binary_classifier_model_selection_criteria = 'cross_entropy_loss')
                                              
```

線形学習の場合、トレーニングデータは protobuf または CSV のコンテンツタイプで受け入れ、推論リクエストは protobuf、CSV、JSON のいずれかのコンテンツタイプで受け入れます。トレーニングデータは特徴量と正解ラベルで構成されるのに対し、推論リクエストのデータは特徴量のみで構成されます。

本番パイプラインでは、データを Amazon SageMaker の protobuf 形式に変換し、Amazon S3 に保存することをお勧めします。ただし、すばやく開始できるように、AWS では、データセットがローカルメモリに収まるほど小さい場合には `record_set` という便利なメソッドを利用して、変換とアップロードを実行できます。このメソッドは、既にお使いのような NumPy 配列に対応しているため、ここではそれを使ってみましょう。`RecordSet` オブジェクトでは、データの一時的な Amazon S3 の場所を記録します。`estimator.record_set` 関数を使用して、トレーニング、検証、テストのレコードを作成します。次に、`estimator.fit` 関数を使用してトレーニングジョブを開始します。

In [ ]:
### Create train, val, test records
train_records = classifier_estimator.record_set(<CODE>,<CODE>, channel='train')# Enter your code here
val_records = classifier_estimator.record_set(<CODE>,<CODE>, channel='validation')# Enter your code here
test_records = classifier_estimator.record_set(<CODE>,<CODE>, channel='test')# Enter your code here

では、アップロードしたデータセットでモデルをトレーニングします。

### サンプルコード
```
linear.fit([train_records,val_records,test_records])
```

In [ ]:
### Fit the classifier
# Enter your code here

## モデルの評価
このセクションでは、トレーニング済みモデルを評価します。まず、`estimator.deploy` 関数で `initial_instance_count= 1` と `instance_type= 'ml.m4.xlarge'` を指定し、モデルを Amazon SageMaker にデプロイします。

In [ ]:
### Deloy an endpoint for batch prediction
classifier_predictor = classifier_estimator.deploy(initial_instance_count=<CODE>,
                                                   instance_type=<CODE>) # Enter your code here

エンドポイントが 'InService' になったら、テストセットでのモデルのパフォーマンスを評価します。`predict_batches` 関数を使用してテストセットでのメトリクスを予測します。

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support

def predict_batches(predictor, features, labels):
    """
    Return evaluation results
    predictor : Predictor object of model
    features: Input features to model
    label: Ground truth target values
    """
    prediction_batches = [predictor.predict(batch) for batch in np.array_split(features, 100)]

    # Parse protobuf responses to extract predicted labels
    extract_label = lambda x: x.label['predicted_label'].float32_tensor.values
    preds = np.concatenate([np.array([extract_label(x) for x in batch]) for batch in prediction_batches])
    preds = preds.reshape((-1,))

    # Calculate accuracy
    accuracy = (preds == labels).sum() / labels.shape[0]
    print(f'Accuracy: {accuracy}')
    
    auc = roc_auc_score(labels, preds)
    print(f'AUC : {auc}')
    
    precision, recall, f1_score, _ = precision_recall_fscore_support(labels, preds, average = 'binary')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1_score: {f1_score}')
    
    confusion_matrix = pd.crosstab(index=labels, columns=np.round(preds), rownames=['True'], colnames=['predictions']).astype(int)
    plt.figure(figsize = (5,5))
    sns.heatmap(confusion_matrix, annot=True, fmt='.2f', cmap="YlGnBu").set_title('Confusion Matrix') 

混同行列を出力するには、テストデータセットで `predict_batches` 関数を実行します。

In [ ]:
# Enter your code here

### 考慮すべき主な質問:
1.テストセットでのモデルのパフォーマンスはトレーニングセットでのパフォーマンスと比べてどのように異なりますか? この比較からどのようなことを推測できますか? 

2.正解率、適合率、再現率などのメトリクスの結果に明らかな違いはありますか? ある場合、そのような違いが見られる理由は何だと思われますか? 

3.ビジネスの状況と目標を考えると、ここで考慮すべき最も重要なメトリクスはどれですか? それはなぜですか?

4.最も重要だと考えるメトリクスの結果は、ビジネスの観点でのニーズを十分に満たすものですか? そうでない場合、次のイテレーション (次の特徴量エンジニアリングのセクション) の際に何を変更できると思いますか? 

以下のセルを使用して、これらの質問や他の質問に答えてください。必要に応じてセルを挿入および削除します。

#### <span style="color: blue;">プロジェクトプレゼンテーション: これらの質問やこのセクションで回答する他の同様の質問に対する回答をプロジェクトプレゼンテーションに記録します。主な詳細と決定事項をプロジェクトプレゼンテーションに記録します。</span>


**問題**: 混同行列からどのようなことを要約できますか?


In [ ]:
# Enter your answer here

## <span style="color:red">ラボ 3 の終わり</span>

ローカルコンピュータにプロジェクトファイルを保存します。次の一連のステップを実行します。

1.ページ上部にある [**File**] メニューをクリックします。

1.[**Download as**] を選択し、[**Notebook(.ipynb)**] をクリックします。 

これにより、現在のノートブックがコンピュータのデフォルトのダウンロードフォルダにダウンロードされます。

# イテレーション 2

# ステップ 4: 特徴量エンジニアリング

これで、モデルのトレーニングと評価の 1 回目のイテレーションを完了しました。モデルで最初に得られた結果はビジネス上の問題を解決するにはおそらく不十分だったと仮定すると、モデルのパフォーマンスを改善できるようにするためにデータに関して何を変更できると思いますか?

### 考慮すべき主な質問:
1.2 つのメインクラス (遅延ありと遅延なし) の均衡はモデルのパフォーマンスにどう影響する可能性がありますか?
2.相関する特徴量はありますか?
3.この段階で、モデルのパフォーマンスに良い影響を与える可能性があり、実行できる特徴量削減の手法はありますか? 
4.データ/データセットの追加を検討できますか?
4.特徴量エンジニアリングを実行した結果、1 回目のイテレーションと比べてモデルのパフォーマンスはどうなりますか?

以下のセルを使用し、上記の質問に従って、モデルのパフォーマンスが改善すると考えられる特徴量エンジニアリングの手法を実行します。必要に応じてセルを挿入および削除します。

#### <span style="color: blue;">プロジェクトプレゼンテーション: 主な決定事項とこのセクションで使用する手法をプロジェクトプレゼンテーションに記録します。また、モデルを再び評価した後に取得する新しいパフォーマンスメトリクスも記録します。</span>

開始する前に、適合率と再現率が約 80% であるのに対し、精度が 99% である理由を考えてみてください。

#### 特徴量を追加する

1.祝日
2.天候

2014 年から 2018 年の祝日はすべてわかっているため、指標変数 **is_holiday** を作成して祝日をマークできます。
例えば、祝日期間中は他の日に比べてフライトの遅延率が高い可能性があるとします。2014～2018 年の祝日を含むブール変数 `is_holiday` を追加します。

In [ ]:
# Source: http://www.calendarpedia.com/holidays/federal-holidays-2014.html

holidays_14 = ['2014-01-01', '2014-01-20', '2014-02-17', '2014-05-26', '2014-07-04', '2014-09-01', '2014-10-13', '2014-11-11', '2014-11-27', '2014-12-25' ] 
holidays_15 = ['2015-01-01', '2015-01-19', '2015-02-16', '2015-05-25', '2015-06-03', '2015-07-04', '2015-09-07', '2015-10-12', '2015-11-11', '2015-11-26', '2015-12-25'] 
holidays_16 = ['2016-01-01', '2016-01-18', '2016-02-15', '2016-05-30', '2016-07-04', '2016-09-05', '2016-10-10', '2016-11-11', '2016-11-24', '2016-12-25', '2016-12-26']
holidays_17 = ['2017-01-02', '2017-01-16', '2017-02-20', '2017-05-29' , '2017-07-04', '2017-09-04' ,'2017-10-09', '2017-11-10', '2017-11-23', '2017-12-25']
holidays_18 = ['2018-01-01', '2018-01-15', '2018-02-19', '2018-05-28' , '2018-07-04', '2018-09-03' ,'2018-10-08', '2018-11-12','2018-11-22', '2018-12-25']
# holidays_19 = ['2019-01-01', '2019-01-21', '2019-02-18', '2019-05-27' , '2019-07-04', '2019-09-02' ,'2019-10-14', '2019-11-11','2019-11-28', '2019-12-25']
holidays = holidays_14+ holidays_15+ holidays_16 + holidays_17+ holidays_18

### Add indicator variable for holidays
data_orig['is_holiday'] = # Enter your code here 

気象データは https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&amp;stations=USW00023174,USW00012960,USW00003017,USW00094846,USW00013874,USW00023234,USW00003927,USW00023183,USW00013881&amp;dataTypes=AWND,PRCP,SNOW,SNWD,TAVG,TMIN,TMAX&amp;startDate=2014-01-01&amp;endDate=2018-12-31 から取得しました。
<br>

このデータセットには、都市の風速、降雨量、積雪、気温に関する情報が空港コード別に含まれています。

**問題**: 降雨、強風、積雪による悪天候がフライトの遅延につながる可能性がありますか? 確認しましょう!

In [ ]:
%%bash

aws s3 cp s3://aws-tc-largeobjects/ILT-TF-200-MLDWTS/flight_delay_project/daily-summaries.csv /home/ec2-user/SageMaker/project/data/
#wget 'https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&amp;stations=USW00023174,USW00012960,USW00003017,USW00094846,USW00013874,USW00023234,USW00003927,USW00023183,USW00013881&amp;dataTypes=AWND,PRCP,SNOW,SNWD,TAVG,TMIN,TMAX&amp;startDate=2014-01-01&amp;endDate=2018-12-31' -O /home/ec2-user/SageMaker/project/data/daily-summaries.csv

空港コードに合わせて作成された気象データをデータセットにインポートします。下記の測候所と空港を分析に使用し、測候所を空港名にマップする `airport` という新しい列を作成します。

In [ ]:
weather = pd.read_csv(<CODE>) # Enter your code here to read 'daily-summaries.csv' file
station = ['USW00023174','USW00012960','USW00003017','USW00094846',
           'USW00013874','USW00023234','USW00003927','USW00023183','USW00013881'] 
airports = ['LAX', 'IAH', 'DEN', 'ORD', 'ATL', 'SFO', 'DFW', 'PHX', 'CLT']

### Map weather stations to airport code
station_map = # Enter your code here 
weather['airport'] = # Enter your code here 

`DATE` 列から `MONTH` というもう 1 つの列を作成します。

In [ ]:
weather['MONTH'] = weather[<CODE>].apply(lambda x: x.split('-')[1])# Enter your code here 
weather.head()

### サンプル出力
```
  STATION DATE AWND PRCP SNOW SNWD TAVG TMAX TMIN airport MONTH
0 USW00023174 2014-01-01 16 0 NaN NaN 131.0 178.0 78.0 LAX 01
1 USW00023174 2014-01-02 22 0 NaN NaN 159.0 256.0 100.0 LAX 01
2 USW00023174 2014-01-03 17 0 NaN NaN 140.0 178.0 83.0 LAX 01
3 USW00023174 2014-01-04 18 0 NaN NaN 136.0 183.0 100.0 LAX 01
4 USW00023174 2014-01-05 18 0 NaN NaN 151.0 244.0 83.0 LAX 01
```

`SNOW` 列と `SNWD` 列を分析し、`fillna()` を使用して欠損値を処理します。`isna()` 関数を使用してすべての列で欠損値があるかどうかを確認します。

In [ ]:
weather.SNOW.fillna(<CODE>, inplace=True)# Enter your code here
weather.SNWD.fillna(<CODE>, inplace=True)# Enter your code here
weather.isna().sum()

**問題**: TAVG、TMAX、TMIN に欠損値がある行のインデックスを出力できますか?

**ヒント**: `isna()` 関数を使用して欠落している行を探し、idx 変数でリストを使用してインデックスを取得します。

In [ ]:
idx = np.array([i for i in range(len(weather))])
TAVG_idx = # Enter your code here 
TMAX_idx = # Enter your code here 
TMIN_idx = # Enter your code here 
TAVG_idx

### サンプル出力

```
array([ 3956, 3957, 3958, 3959, 3960, 3961, 3962, 3963, 3964,
        3965, 3966, 3967, 3968, 3969, 3970, 3971, 3972, 3973,
        3974, 3975, 3976, 3977, 3978, 3979, 3980, 3981, 3982,
        3983, 3984, 3985, 4017, 4018, 4019, 4020, 4021, 4022,
        4023, 4024, 4025, 4026, 4027, 4028, 4029, 4030, 4031,
        4032, 4033, 4034, 4035, 4036, 4037, 4038, 4039, 4040,
        4041, 4042, 4043, 4044, 4045, 4046, 4047, 13420])
```

欠落している TAVG、TMAX、TMIN に特定の測候所/空港の平均値を代入できます。TAVG_idx で連続する行が欠落しているため、直前の値を代入することはできません。代わりに、平均値を代入します。`groupby` 関数を使用して、平均値を含む変数を集計します。

In [ ]:
weather_impute = weather.groupby([<CODE>]).agg({'TAVG':'mean','TMAX':'mean', 'TMIN':'mean' }).reset_index()# Enter your code here
weather_impute.head(2)

平均値データを気象データとマージします。

In [ ]:
### get the yesterday's data
weather = pd.merge(weather, weather_impute, how='left', left_on=['MONTH','STATION'], right_on = ['MONTH','STATION'])\
.rename(columns = {'TAVG_y':'TAVG_AVG',
                   'TMAX_y':'TMAX_AVG',
                   'TMIN_y':'TMIN_AVG',
                   'TAVG_x':'TAVG',
                   'TMAX_x':'TMAX',
                   'TMIN_x':'TMIN'})

欠損値があるかどうかをもう一度確認します。

In [ ]:
weather.TAVG[TAVG_idx] = weather.TAVG_AVG[TAVG_idx]
weather.TMAX[TMAX_idx] = weather.TMAX_AVG[TMAX_idx]
weather.TMIN[TMIN_idx] = weather.TMIN_AVG[TMIN_idx]
weather.isna().sum()

データセットから `STATION、MONTH、TAVG_AVG、TMAX_AVG、TMIN_AVG、TMAX、TMIN、SNWD` を削除します。

In [ ]:
weather.drop(columns=['STATION','MONTH','TAVG_AVG', 'TMAX_AVG', 'TMIN_AVG', 'TMAX' ,'TMIN', 'SNWD'],inplace=True)

データセットに出発地と目的地の気象条件を追加します。

In [ ]:
### Add origin weather conditions
data_orig = pd.merge(data_orig, weather, how='left', left_on=['FlightDate','Origin'], right_on = ['DATE','airport'])\
.rename(columns = {'AWND':'AWND_O','PRCP':'PRCP_O', 'TAVG':'TAVG_O', 'SNOW': 'SNOW_O'})\
.drop(columns=['DATE','airport'])

### Add destination weather conditions
data_orig = pd.merge(data_orig, weather, how='left', left_on=['FlightDate','Dest'], right_on = ['DATE','airport'])\
.rename(columns = {'AWND':'AWND_D','PRCP':'PRCP_D', 'TAVG':'TAVG_D', 'SNOW': 'SNOW_D'})\
.drop(columns=['DATE','airport'])

**注意**: 結合後に null/NA を確認することをお勧めします。

In [ ]:
sum(data.isna().any())

In [ ]:
data_orig.columns

ワンホットエンコーディングを使用してカテゴリデータを数値データに変換します。

In [ ]:
data = data_orig.copy()
data = data[['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Reporting_Airline', 'Origin', 'Dest','Distance','DepHourofDay', 'is_delay', 'is_holiday', 'AWND_O', 'PRCP_O',
       'TAVG_O', 'AWND_D', 'PRCP_D', 'TAVG_D', 'SNOW_O', 'SNOW_D']]


categorical_columns = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek',
       'Reporting_Airline', 'Origin', 'Dest', 'is_holiday','is_delay']
for c in categorical_columns:
    data[c] = data[c].astype('category')

In [ ]:
data_dummies = # Enter your code here

### サンプルコード

```
data_dummies = pd.get_dummies(data[['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'Reporting_Airline', 'Origin', 'Dest', 'is_holiday']], drop_first=True)
data = pd.concat([data, data_dummies], axis = 1)
categorical_columns.remove('is_delay')
data.drop(categorical_columns,axis=1, inplace=True)
```

新しい列を確認します。

In [ ]:
data.columns

### サンプル出力

```
Index(['Distance', 'DepHourofDay', 'is_delay', 'AWND_O', 'PRCP_O', 'TAVG_O',
       'AWND_D', 'PRCP_D', 'TAVG_D', 'SNOW_O', 'SNOW_D', 'Year_2015',
       'Year_2016', 'Year_2017', 'Year_2018', 'Quarter_2', 'Quarter_3',
       'Quarter_4', 'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6',
       'Month_7', 'Month_8', 'Month_9', 'Month_10', 'Month_11', 'Month_12',
       'DayofMonth_2', 'DayofMonth_3', 'DayofMonth_4', 'DayofMonth_5',
       'DayofMonth_6', 'DayofMonth_7', 'DayofMonth_8', 'DayofMonth_9',
       'DayofMonth_10', 'DayofMonth_11', 'DayofMonth_12', 'DayofMonth_13',
       'DayofMonth_14', 'DayofMonth_15', 'DayofMonth_16', 'DayofMonth_17',
       'DayofMonth_18', 'DayofMonth_19', 'DayofMonth_20', 'DayofMonth_21',
       'DayofMonth_22', 'DayofMonth_23', 'DayofMonth_24', 'DayofMonth_25',
       'DayofMonth_26', 'DayofMonth_27', 'DayofMonth_28', 'DayofMonth_29',
       'DayofMonth_30', 'DayofMonth_31', 'DayOfWeek_2', 'DayOfWeek_3',
       'DayOfWeek_4', 'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_7',
       'Reporting_Airline_DL', 'Reporting_Airline_OO', 'Reporting_Airline_UA',
       'Reporting_Airline_WN', 'Origin_CLT', 'Origin_DEN', 'Origin_DFW',
       'Origin_IAH', 'Origin_LAX', 'Origin_ORD', 'Origin_PHX', 'Origin_SFO',
       'Dest_CLT', 'Dest_DEN', 'Dest_DFW', 'Dest_IAH', 'Dest_LAX', 'Dest_ORD',
       'Dest_PHX', 'Dest_SFO', 'is_holiday_1'],
      dtype='object')
```

列名を `is_delay` から `target` に戻します。前と同じコードを使用します。

In [ ]:
data.rename(columns = {<CODE>:<CODE>}, inplace=True )# Enter your code here

トレーニングセットをもう一度作成します。

In [ ]:
# Enter your code here

### 新しいベースライン分類器

ここで、上記の新しい特徴量によりモデルの予測検出力が強化されたかどうかを確認します。

In [ ]:
# Instantiate the LinearLearner estimator object
num_classes = # Enter your code here
classifier_estimator = # Enter your code here

### サンプルコード

```
num_classes = len(pd.unique(train_labels)) 
classifier_estimator = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                               instance_count=1,
                                               instance_type='ml.m4.xlarge',
                                               predictor_type='binary_classifier',
                                              binary_classifier_model_selection_criteria = 'cross_entropy_loss')
```

In [ ]:
train_records = classifier_estimator.record_set(train_features, train_labels, channel='train')
val_records = classifier_estimator.record_set(val_features, val_labels, channel='validation')
test_records = classifier_estimator.record_set(test_features, test_labels, channel='test')

classifier_estimator.fit([train_records, val_records, test_records])

In [ ]:
classifier_predictor = # Enter your code here

In [ ]:
predict_batches(classifier_predictor, test_features, test_labels)

線形モデルから、パフォーマンスが少ししか改善されなかったことがわかります。Amazon SageMaker で XGBoost というツリーベースのアンサンブルモデルを試します。

### XGBoost モデルを試す

以下の手順を行う必要があります。  

1.トレーニングセット変数を使用します。それらの変数を CSV ファイル: train.csv、validation.csv、test.csv として保存します。  
2.変数にバケット名を格納します。Amazon S3 バケット名は、ラボ手順の左側に表示されます。 
a. `bucket = <LabBucketName>`  
b. `prefix = 'sagemaker/xgboost'`  
3.Boto3 を使用してバケットにモデルをアップロードします。   

In [ ]:
train_data, validation_data, test_data = np.split(data.sample(frac=1, random_state=1729), [int(0.7 * len(data)), int(0.9*len(data))])  

pd.concat([train_data['target'], train_data.drop(['target'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
pd.concat([validation_data['target'], validation_data.drop(['target'], axis=1)], axis=1).to_csv('validation.csv', index=False, header=False)
pd.concat([test_data['target'], test_data.drop(['target'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)

**`<LabBucketName>`** をラボアカウントに用意されたリソース名に置き換えます。

In [ ]:
# Enter your code here to define bucket and prefix
bucket = <LabBucketName> # Enter your code here
prefix = 'sagemaker/xgboost'

In [ ]:
### Upload data to S3
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

In [ ]:
from sagemaker.amazon.amazon_estimator import image_uris
container = image_uris.retrieve('xgboost',boto3.Session().region_name, '0.90-1')

`sagemaker.s3_input` 関数を使用して、トレーニングと検証のデータセット用の record_set を作成します。

In [ ]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

In [ ]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role = sagemaker.get_execution_role(),
                                    instance_count=1,
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        eval_metric = "auc",
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

新しいモデルの予測器をデプロイし、テストデータセットでモデルを評価します。

In [ ]:
from sagemaker.serializers import CSVSerializer

xgb_predictor = xgb.deploy(initial_instance_count = <CODE>,
                           instance_type = <CODE>,
                          serializer=CSVSerializer())# Enter your code here

In [ ]:
test_data = pd.concat([test_data['target'], test_data.drop(['target'], axis=1)], axis=1)

In [ ]:
def predict(predictor , features, labels , prob_threshold = 0.5, rows=500):
    """
    Return evaluation results
    predictor : Predictor object of model
    features: Input features to model
    label: Ground truth target values
    prob_threshold : Probability cut off to separate positive and negative class
    """
    split_array = np.array_split(features, int(features.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    preds = np.fromstring(predictions[1:], sep=',')
    preds = preds.reshape((-1,))
    predictions = np.where(preds > prob_threshold , 1, 0)
    labels = labels.reshape((-1,))
    
    
    # Calculate accuracy
    accuracy = (predictions == labels).sum() / labels.shape[0]
    print(f'Accuracy: {accuracy}')
    
    auc = roc_auc_score(labels, preds)
    print(f'AUC : {auc}')
    
    precision, recall, f1_score, _ = precision_recall_fscore_support(labels, predictions, average = 'binary')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1_score: {f1_score}')
    
    
    confusion_matrix = pd.crosstab(index=labels, columns=np.round(preds), rownames=['True'], colnames=['predictions']).astype(int)
    plt.figure(figsize = (5,5))
    sns.heatmap(confusion_matrix, annot=True, fmt='.2f', cmap="YlGnBu").set_title('Confusion Matrix') 
    return list(preds)

In [ ]:
predictions = predict(xgb_predictor, test_data.as_matrix()[:, 1:] , test_data.iloc[:, 0:1].as_matrix(), prob_threshold = 0.5)

### さまざまなしきい値を試す

In [ ]:
predictions = predict(xgb_predictor, test_data.as_matrix()[:, 1:] , test_data.iloc[:, 0:1].as_matrix(), prob_threshold = 0.8)

**問題**: テストセットでのモデルのパフォーマンスからどのような結論を導くことができますか?

In [ ]:
#Enter your answer here

### ハイパーパラメータの最適化 (HPO)

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

### You can spin up multiple instances to do hyperparameter optimization in parallel

xgb = sagemaker.estimator.Estimator(container,
                                    role=sagemaker.get_execution_role(),
                                    instance_count= 2, # make sure you have limit set for these instances
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)

xgb.set_hyperparameters(eval_metric='auc',
                        objective='binary:logistic',
                        num_round=100,
                        rate_drop=0.3,
                        tweedie_variance_power=1.4)

In [ ]:
hyperparameter_ranges = {'eta': ContinuousParameter(0, 0.5),
                        'min_child_weight': ContinuousParameter(3, 10),
                         'num_round': IntegerParameter(10, 150),
                        'alpha': ContinuousParameter(0, 2),
                        'max_depth': IntegerParameter(10, 15)}

objective_metric_name = 'validation:auc'

tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=10,
                            max_parallel_jobs=3)

In [ ]:
tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

In [ ]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

<i class="fas fa-exclamation-triangle" style="color:red"></i> トレーニングジョブが終了するまで待ちます。25～30 分ほどかかります。

**ハイパーパラメータの最適化ジョブをモニタリングするには、以下を行います。**  

1.AWS マネジメントコンソールの [**サービス**] メニューで [**Amazon SageMaker**] をクリックします。 
1.[**Training**] > [**Hyperparameter tuning jobs**] の順にクリックします。 
1.各ハイパーパラメータチューニングジョブのステータス、そのジョブの目標メトリクス値とログを確認できます。 

In [ ]:
sage_client = boto3.Session().client('sagemaker')
tuning_job_name = tuner.latest_tuning_job.job_name

# Run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("%d training jobs have completed" % job_count)
    
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']


from pprint import pprint
if tuning_job_result.get('BestTrainingJob',None):
    print("Best model found so far:")
    pprint(tuning_job_result['BestTrainingJob'])
else:
    print("No training jobs have reported results yet.")
    
### Get the hyperparameter tuning job results in a dataframe
tuner_df = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name).dataframe()

if len(tuner_df) > 0:
    df = tuner_df[tuner_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        pd.set_option('display.max_colwidth', -1) # Don't truncate TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")
        
tuner_df

ハイパーパラメータの最適化トレーニングから得られた、精度が最も良いモデルをデプロイします。

In [ ]:
xgb_predictor_hpo = # Enter your code here.

In [ ]:
predictions = predict(xgb_predictor_hpo, test_data.as_matrix()[:, 1:] , test_data.iloc[:, 0:1].as_matrix(), prob_threshold = 0.5)

**問題**: 別のハイパーパラメータやハイパーパラメータ範囲を試してください。モデルが改善されますか?

### 特徴量の重要度

すべてのハイパーパラメータチューニングジョブのモデルファイルは、上記の表にあるトレーニングジョブ名で、フォルダ: "{bucket}/{prefix}/output/" に保存されます。モデルをロードし、通常の sklearn モデルオブジェクトのようにデータの解釈に使用できます。

In [ ]:
#best_hpo_model_path = "s3://" + bucket + "/sagemaker/xgboost/output/<Best model TrainingJobName>/output/model.tar.gz"
best_hpo_model_path = <path-to-your-model>
### Download best_hpo_model_path to local
!aws s3 cp {best_hpo_model_path} .

In [ ]:
# Install xgboost
!pip install xgboost=='0.90'

#### 選択したモデルファイルをロードする

In [ ]:
import pickle
import tarfile
import xgboost

with open('model.tar.gz', 'rb') as f:
    with tarfile.open(fileobj=f, mode='r') as tar_f:
        with tar_f.extractfile('xgboost-model') as extracted_f:
            xgbooster = pickle.load(extracted_f)

#### XGBoost モデルに列名をマップする

In [ ]:
columns = list(data.columns)
columns.remove('target')

In [ ]:
feature_importance = xgbooster.get_fscore()
feature_importance_col = {}

for column, fname in zip(columns, xgbooster.feature_names):
    try:
         feature_importance_col[column] = feature_importance[fname]
    except Exception:
        pass

#### 特徴量の重要度値で並べ替える

In [ ]:
sorted(feature_importance_col.items(), key=lambda kv: kv[1], reverse=True)

### サンプル出力

```
[('AWND_O', 13851),
 ('AWND_D', 13452),
 ('DepHourofDay', 13344),
 ('TAVG_O', 13106),
 ('TAVG_D', 12800),
 ('Distance', 8478),
 ('PRCP_O', 4210),
 ('PRCP_D', 3916),
 ('Reporting_Airline_UA', 1791),
 ('Year_2016', 1290),
 ('Year_2015', 1285),
 ('Year_2018', 1157),
 ('Quarter_4', 1092),
 ('Year_2017', 1009),
 ('DayOfWeek_5', 838),
 ('DayOfWeek_4', 833),
 ('Quarter_2', 799),
 ('DayOfWeek_7', 783),
 ('Reporting_Airline_WN', 736),
 ('DayOfWeek_3', 718),
 ('Origin_ORD', 706),
 ('DayOfWeek_2', 685),
 ('Reporting_Airline_DL', 663),
 ('Dest_LAX', 627),
 ('DayOfWeek_6', 614),
 ('Reporting_Airline_OO', 596),
 ('Origin_LAX', 590),
 ('Month_11', 565),
 ('Month_5', 543),
 ('Dest_ORD', 539),
 ('SNOW_O', 506),
 ('Month_8', 497),
 ('Month_12', 495),
 ('Month_7', 484),
 ('Origin_DFW', 480),
 ('Quarter_3', 477),
 ('Dest_DFW', 462),
 ('Origin_DEN', 456),
 ('Origin_SFO', 438),
 ('Month_6', 437),
 ('Dest_SFO', 437),
 ('Month_10', 422),
 ('Month_9', 407),
 ('Month_4', 394),
 ('Dest_DEN', 379),
 ('SNOW_D', 375),
 ('Month_3', 369),
 ('Dest_IAH', 369),
 ('Origin_PHX', 347),
 ('Dest_PHX', 346),
 ('Origin_IAH', 342),
 ('DayofMonth_15', 324),
 ('DayofMonth_17', 322),
 ('DayofMonth_21', 322),
 ('Origin_CLT', 322),
 ('DayofMonth_20', 316),
 ('DayofMonth_9', 313),
 ('DayofMonth_19', 307),
 ('DayofMonth_26', 303),
 ('DayofMonth_7', 301),
 ('DayofMonth_4', 300),
 ('DayofMonth_2', 299),
 ('DayofMonth_27', 298),
 ('DayofMonth_3', 294),
 ('DayofMonth_28', 287),
 ('Month_2', 284),
 ('DayofMonth_5', 284),
 ('DayofMonth_12', 282),
 ('DayofMonth_11', 276),
 ('Dest_CLT', 276),
 ('DayofMonth_6', 275),
 ('DayofMonth_22', 274),
 ('DayofMonth_8', 273),
 ('DayofMonth_18', 273),
 ('DayofMonth_29', 267),
 ('DayofMonth_23', 266),
 ('DayofMonth_14', 263),
 ('DayofMonth_30', 260),
 ('DayofMonth_10', 259),
 ('DayofMonth_16', 257),
 ('DayofMonth_24', 255),
 ('DayofMonth_13', 245),
 ('DayofMonth_25', 237),
 ('is_holiday_1', 231),
 ('DayofMonth_31', 164)]
```

上記の特徴量の重要度に基づき、出発地と目的地の両方における DepHourofDay、Airwind、Temperature がフライトの遅延を判断するうえで重要な要因であることがわかります。

これは、特徴量に関して人間が直感的に理解できるものと、モデルが実際に学習するものを評価するための 1 つの方法です。

## まとめ

モデルのトレーニングと評価のイテレーションを何回か行いました。ここで、もう少し時間があることを想定して、このプロジェクトについてまとめ、これまでに学んだこと、今後どのような手順を実行するのかについてよく考えます。以下のセルを使用して、これらの質問や他の関連する質問に回答してください。

1.モデルのパフォーマンスはビジネス目標を満たしていますか? 満たしていない場合、チューニングにさらに多くの時間をかけられるとしたら、別にやってみたいことが何かありますか?
2.データセット、特徴量、ハイパーパラメータを変更すると、モデルはどの程度改善しましたか? このプロジェクト全体で採用した手法のうち、モデルが最も改善されたと感じたのはどのような種類の手法ですか?
3.このプロジェクト全体で直面した最大の課題は何でしたか?
4.パイプラインのフェーズに関する質問のうち、理解できていないままのものがありますか?
5.このプロジェクトで機械学習について学んだ最も重要なことは何ですか? (3 つ挙げてください)

#### <span style="color: blue;">プロジェクトプレゼンテーション: この質問に対する自分の回答の要約をプロジェクトプレゼンテーションに含めます。この時点でプロジェクトプレゼンテーションに使用するメモをすべてまとめ、クラスで結果を発表する準備をします。</span>